<a href="https://colab.research.google.com/github/ApoloXO/OR/blob/main/Queueing/discrete_event_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Queueing Theory Intro
## Discrete Event Simulation
Carlos Alexander Grajales Correa \\
Professor Universidad de Antioquia, Colombia \\
alexander.grajales@udea.edu.co \\
**Reference:**
*This notebook contains code examples referring to the book*

"Applied Mathematics with Open-Source Software: Operational Research Problems
with Python and R". Chapter: Discrete Event Simulation.   Vincent Knight and Geraint Palmer.  CRC Press Taylor and Francis Group, 2022.*


\\


☝ Before start

At Google Colab, for this intro, you will first need to install

* ciw
___

In [ ]:
!pip install ciw
import ciw

# Bicycle Repair Shop Problem

A bicycle repair shop would like to reconfigure in order to guarantee that all bicycles processed take a maximum of 30 minutes. Their current set-up is as follows:

* Bicycles arrive randomly at the shop at a rate of 15 per hour;
* they wait in line to be seen at an inspection counter, staffed by one member of staff who can inspect one bicycle at a time. On average an inspection takes around 3 minutes;
* around 20% of bicycles do not need repair after inspection, and they are then ready for collection;
* around 80% of bicycles go on to be repaired after inspection. These then wait in line outside the repair workshop, which is staffed by two members of staff who can each repair one bicycle at a time. On average a repair takes around 6 minutes;
* after repair the bicycles are ready for collection.

An assumption of infinite capacity at the bicycle repair shop for waiting bicycles is made.The shop will hire an extra member of staff in order to meet their target of a maximum time in the system of 30 minutes. They would like to know if they should work on the inspection counter or in the repair workshop.

___

☝ First we define a function that gives a Network object, containing the structure of the repair shop.

In [2]:
def build_network_object(
    num_inspectors=1,
    num_repairers=2,
):
    """Returns a Network object that defines the repair shop.

    Args:
        num_inspectors: a positive integer (default: 1)
        num_repairers: a positive integer (default: 2)

    Returns:
        a Ciw network object
    """
    arrival_rate = 15
    inspection_rate = 20
    repair_rate = 10
    prob_need_repair = 0.8
    N = ciw.create_network(
        arrival_distributions=[
            ciw.dists.Exponential(arrival_rate),
            None,
            #ciw.dists.NoArrivals(),
        ],
        service_distributions=[
            ciw.dists.Exponential(inspection_rate),
            ciw.dists.Exponential(repair_rate),
        ],
        number_of_servers=[num_inspectors, num_repairers],
        routing=[[0.0, prob_need_repair], [0.0, 0.0]],
    )
    return N

☝ We can see information such as number of nodes in the network:

In [ ]:
N = build_network_object()
print(N.number_of_nodes)

☝ Then we define a function that runs one trial of the simulation.

In [4]:
def run_simulation(network, seed=0):
    """Builds a simulation object and runs it for 8 time units.

    Args:
        network: a Ciw network object
        seed: a float (default: 0)

    Returns:
        a Ciw simulation object after a run of the simulation
    """
    max_time = 8
    ciw.seed(seed)
    Q = ciw.Simulation(network)
    Q.simulate_until_max_time(max_time)
    return Q

☝ From one trial we can obtain the proportion of bicycles taking over half an hour:

In [5]:
import pandas as pd


def get_proportion(Q):
    """Returns the proportion of bicycles spending over a given
    limit at the repair shop.

    Args:
        Q: a Ciw simulation object after a run of the
           simulation

    Returns:
        a real
    """
    limit = 0.5
    inds = Q.nodes[-1].all_individuals
    recs = pd.DataFrame(
        dr for ind in inds for dr in ind.data_records
    )
    recs["total_time"] = recs["exit_date"] - recs["arrival_date"]
    total_times = recs.groupby("id_number")["total_time"].sum()
    return (total_times > limit).mean()

☝ Putting all this together for one trial

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

N = build_network_object()
Q = run_simulation(N)
p = get_proportion(Q)

recs = Q.get_all_records()
recs_pd = pd.DataFrame(recs)
recs_pd.to_excel("recsExcel.xlsx")

waits = [r.waiting_time for r in recs]
waits
plt.hist(waits);

☝ A function to find the average proportion over a number of trials

In [21]:
def get_average_proportion(num_inspectors=1, num_repairers=2):
    """Returns the average proportion of bicycles spending over a
    given limit at the repair shop.

    Args:
        num_inspectors: a positive integer (default: 1)
        num_repairers: a positive integer (default: 2)

    Returns:
        a real
    """
    num_trials = 100
    N = build_network_object(
        num_inspectors=num_inspectors,
        num_repairers=num_repairers,
    )
    proportions = []
    for trial in range(num_trials):
        Q = run_simulation(N, seed=trial)
        proportion = get_proportion(Q=Q)
        proportions.append(proportion)
    return sum(proportions) / num_trials


☝ The proportion with current staff:

In [ ]:
p = get_average_proportion(num_inspectors=1, num_repairers=2)
print(round(p, 6))

☝ The proportion with an extra inspector:

In [ ]:
p = get_average_proportion(num_inspectors=2, num_repairers=2)
print(round(p, 6))

☝ The proportion with an extra repairer:

In [ ]:
p = get_average_proportion(num_inspectors=1, num_repairers=3)
print(round(p, 6))

☝ The proportion with an extra inspector and extra repairer:

In [ ]:
p = get_average_proportion(num_inspectors=2, num_repairers=3)
print(round(p, 6))